In [2]:
# !pip install torch

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the RNN model
class CustomLanguageModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(CustomLanguageModel, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input.view(1, -1))
        output, hidden = self.rnn(embedded, hidden)
        output = self.fc(output.view(1, -1))
        return output, hidden

    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size)

def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        try:
            tensor[c] = all_characters.index(string[c])
        except ValueError:
            # Skip characters not found in all_characters
            continue
    return tensor


def train(model, input_sequence, target_sequence, optimizer, criterion):
    hidden = model.init_hidden()
    optimizer.zero_grad()
    loss = 0

    for i in range(input_sequence.size(0)):
        output, hidden = model(input_sequence[i], hidden)
        loss += criterion(output, target_sequence[i].unsqueeze(0))

    loss.backward()
    optimizer.step()

    return loss.item() / input_sequence.size(0)

def generate_text(model, start_sequence, length):
    with torch.no_grad():
        input_sequence = char_tensor(start_sequence)
        hidden = model.init_hidden()

        for i in range(len(start_sequence) - 1):
            _, hidden = model(input_sequence[i], hidden)

        output_sequence = start_sequence

        for i in range(length):
            output, hidden = model(input_sequence[-1], hidden)
            predicted_char = torch.argmax(output).item()
            output_sequence += all_characters[predicted_char]
            input_sequence = char_tensor(all_characters[predicted_char])

        return output_sequence

# Prepare the data
all_characters = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789,.!?'"
n_characters = len(all_characters)
n_hidden = 128

data = "Once upon a time, there was a little girl named Red Riding Hood. She lived in a small village."

# Set up the model, criterion, and optimizer
model = CustomLanguageModel(n_characters, n_hidden, n_characters)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 1000
print_every = 100
losses = []

for epoch in range(1, num_epochs + 1):
    input_sequence = char_tensor(data[:-1])
    target_sequence = char_tensor(data[1:])
    loss = train(model, input_sequence, target_sequence, optimizer, criterion)
    losses.append(loss)

    if epoch % print_every == 0:
        print(f"[{epoch}/{num_epochs}] Loss: {loss:.4f}")

# Generate text using the trained model
start_sequence = "Once upon a time"
generated_text = generate_text(model, start_sequence, length=100)
print(generated_text)


[100/1000] Loss: 0.1711
[200/1000] Loss: 0.0236
[300/1000] Loss: 0.0100
[400/1000] Loss: 0.0058
[500/1000] Loss: 0.0039
[600/1000] Loss: 0.0028
[700/1000] Loss: 0.0021
[800/1000] Loss: 0.0017
[900/1000] Loss: 0.0013
[1000/1000] Loss: 0.0011
Once upon a time,athereawasaaalittleagirlanamedaRedaRidingaHood.aShealivedainaaasmallavillage.aShealivedainaaasmalla


In [11]:
start_sequence = "Once"
generated_text = generate_text(model, start_sequence, length=100)
print(generated_text)


Onceauponaaatime,athereawasaaalittleagirlanamedaRedaRidingaHood.aShealivedainaaasmallavillage.aShealived
